# Forest Inference Library (FIL)
The forest inference library is used to load saved forest models of xgboost, lightgbm and perform inference on them. It can be used to perform both classification and regression. In this notebook, we'll begin by fitting a model with XGBoost and saving it. We'll then load the saved model into FIL and use it to infer on new data.

FIL works in the same way with lightgbm model as well.

The model accepts both numpy arrays and cuDF dataframes. In order to convert your dataset to cudf format please read the cudf documentation on https://docs.rapids.ai/api/cudf/stable. 

For additional information on the forest inference library please refer to the documentation on https://docs.rapids.ai/api/cuml/stable/api.html#forest-inferencing

In [1]:
%load_ext memory_profiler

In [2]:
%%memit
import cupy
import os

from cuml.test.utils import array_equal
from cuml.common.import_utils import has_xgboost

from cuml.datasets import make_classification
from cuml.metrics import accuracy_score
from cuml.model_selection import train_test_split
    
from cuml import ForestInference

peak memory: 338.53 MiB, increment: 288.77 MiB


### Check for xgboost
Checks if xgboost is present, if not then it throws an error.

In [3]:
%%memit
if has_xgboost():
    import xgboost as xgb
else:
    raise ImportError("Please install xgboost using the conda package,"
                      "e.g.: conda install -c conda-forge xgboost")

peak memory: 344.24 MiB, increment: 5.45 MiB


## Define parameters

In [18]:
%%memit
# synthetic data size
n_rows = 2000
n_columns = 100
n_categories = 2
random_state = cupy.random.RandomState(43210)

# fraction of data used for model training
train_size = 0.5

# trained model output filename
model_path = 'xgb_80.model'

peak memory: 2386.93 MiB, increment: 0.00 MiB


## Generate data

In [12]:
%%memit
# create the dataset
X, y = make_classification(
    n_samples=n_rows,
    n_features=n_columns,
    n_informative=int(n_columns/5),
    n_classes=n_categories,
    random_state=42
)

# convert the dataset to float32
X = X.astype('float32')
y = y.astype('float32')

# split the dataset into training and validation splits
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size)

peak memory: 2386.87 MiB, increment: 0.08 MiB


## Load Forest Inference Library (FIL)

The load function of the ForestInference class accepts the following parameters:

       filename : str
           Path to saved model file in a treelite-compatible format
           (See https://treelite.readthedocs.io/en/latest/treelite-api.html
        output_class : bool
           If true, return a 1 or 0 depending on whether the raw prediction
           exceeds the threshold. If False, just return the raw prediction.
        threshold : float
           Cutoff value above which a prediction is set to 1.0
           Only used if the model is classification and output_class is True
        algo : string name of the algo from (from algo_t enum)
             'NAIVE' - simple inference using shared memory
             'TREE_REORG' - similar to naive but trees rearranged to be more
                              coalescing-friendly
             'BATCH_TREE_REORG' - similar to TREE_REORG but predicting
                                    multiple rows per thread block
        model_type : str
            Format of saved treelite model to load.
            Can be 'xgboost', 'lightgbm'

## Loaded the saved model
Use FIL to load the saved xgboost model

In [17]:
%%time
%%memit


fil_model_naive = ForestInference.load(
    filename=model_path,
    algo='NAIVE',
    output_class=True,
    threshold=0.50,
    model_type='xgboost'
)

peak memory: 2386.93 MiB, increment: 0.00 MiB
CPU times: user 516 ms, sys: 68.5 ms, total: 584 ms
Wall time: 329 ms


In [20]:
%%time
%%memit

fil_model_btr = ForestInference.load(
    filename=model_path,
    algo='BATCH_TREE_REORG',
    output_class=True,
    threshold=0.50,
    model_type='xgboost'
)


peak memory: 4946.93 MiB, increment: 2560.00 MiB
CPU times: user 3.43 s, sys: 1.17 s, total: 4.6 s
Wall time: 4.64 s


## Predict using FIL

In [6]:
%%time
%%memit
# perform prediction on the model loaded from path
fil_preds_naive = fil_model_naive.predict(X_validation)

NameError: name 'X_validation' is not defined

In [9]:
%%time
%%memit
# perform prediction on the model loaded from path
fil_preds_btr = fil_model_btr.predict(X_validation)

peak memory: 2366.82 MiB, increment: 0.01 MiB
CPU times: user 200 ms, sys: 69.3 ms, total: 269 ms
Wall time: 402 ms


In [10]:
len(fil_preds_naive)

1000000